## Scraping Multiple Pages - Las Vegas

#### Importing Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.parse
import pandas as pd

#### Scraping the data

In [2]:
address = []
bedrooms = []
bathrooms = []
sqft = []
year_built = []
parking = []
pool = []
price = []

# Url part I
url_1 = 'https://www.trulia.com'

for i in range(1, 26):
    
    #website
    www = 'https://www.trulia.com/NV/Las_Vegas/' +str(i) +'_p/'
    
    #requests
    response = requests.get(www)
    
    # soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    #result container
    result_container = soup.find_all('li', {'class':'SearchResultsList__WideCell-b7y9ki-2'})
    
    #update results
    results_update = []
    
    # only results with attribute 'data-testeid'
    for r in result_container:
        if r.has_attr('data-testid'):
            results_update.append(r)
        
    #relative url
    relative_url = []
    
    #loop through results
    for i in results_update:
        for j in i.find_all('div', {'data-testid':'property-card-details'}):
            relative_url.append(j.find('a').get('href'))
            
    #empty list url_joined
    url_joined = []

    for i in relative_url:
        url_joined.append(urllib.parse.urljoin(url_1, i))
        
       
    # loop through all joined links
    for i in url_joined:
        response = requests.get(i)

        #create soup object
        soup = BeautifulSoup(response.content, 'html.parser')

        #address
        try:
            address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
        except:
            address.append('')

        #bedrooms
        try:
            bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
        except:
            bedrooms.append('')

        #bathrooms
        try:
            bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
        except:
            bathrooms.append('')

        #sqft
        try:
            sqft.append(soup.find('li', {'data-testid':'floor'}).get_text())
        except:
            sqft.append('')

        #year built
        try:
            year_built.append(soup.find('div', string='Year Built').findNext('div').get_text())
        except:
            year_built.append('')

         #parking
        try:
            parking.append(soup.find('div', string='Parking').findNext('div').get_text())
        except:
            parking.append('')

         #pool
        try:
            pool.append(soup.find('div', string='Pool').findNext('div').get_text())
        except:
            pool.append('')

         #price
        try:
            price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
        except:
            price.append('')

        #creating a dictionary with the results
        output = {'Address': address, 'Bedrooms':bedrooms, 'Bathrooms': bathrooms, 'Area': sqft, 'Year Built': year_built,
                 'Parking': parking, 'Pool': pool, 'Price': price}

In [3]:
df = pd.DataFrame(output)
df

,Address,Bedrooms,Bathrooms,Area,Year Built,Parking,Pool,Price
0,6935 Smiling Cloud Ave,2 Beds,3 Baths,"1,209 sqft",2007,Garage,No,"$272,100"
1,9828 Piper Glen Pl,3 Beds,2 Baths,"1,122 sqft",1997,Garage,No,"$389,900"
2,7904 Aspect Way,4 Beds,3 Baths,"2,625 sqft",1995,Garage,Yes,"$556,900"
3,4024 Grasmere Ave,3 Beds,3 Baths,"1,476 sqft",1986,Garage,,"$292,900"
4,3829 Pumpkin Creek St,4 Beds,3 Baths,"2,315 sqft",2005,Garage,No,"$411,300"
...,...,...,...,...,...,...,...,...
995,1708 Ivanhoe Way,3 Beds,2 Baths,"2,095 sqft",1963,1 Carport Spaces,No,"$495,000"
996,3975 N Hualapai Way #159,2 Beds,2 Baths,"1,110 sqft",2007,No Info,Pool,"$255,000"
997,296 Molinetto St,3 Beds,3 Baths,"2,095 sqft",2019,2 Car Garage,Yes,"$995,000"
998,816 Trotter Cir,5 Beds,5 Baths,"5,357 sqft (on 0.35 acres)",1989,Garage,Yes,"$1,995,000"


In [4]:
df['Location'] = 'Las Vegas'
df

,Address,Bedrooms,Bathrooms,Area,Year Built,Parking,Pool,Price,Location
0,6935 Smiling Cloud Ave,2 Beds,3 Baths,"1,209 sqft",2007,Garage,No,"$272,100",Las Vegas
1,9828 Piper Glen Pl,3 Beds,2 Baths,"1,122 sqft",1997,Garage,No,"$389,900",Las Vegas
2,7904 Aspect Way,4 Beds,3 Baths,"2,625 sqft",1995,Garage,Yes,"$556,900",Las Vegas
3,4024 Grasmere Ave,3 Beds,3 Baths,"1,476 sqft",1986,Garage,,"$292,900",Las Vegas
4,3829 Pumpkin Creek St,4 Beds,3 Baths,"2,315 sqft",2005,Garage,No,"$411,300",Las Vegas
...,...,...,...,...,...,...,...,...,...
995,1708 Ivanhoe Way,3 Beds,2 Baths,"2,095 sqft",1963,1 Carport Spaces,No,"$495,000",Las Vegas
996,3975 N Hualapai Way #159,2 Beds,2 Baths,"1,110 sqft",2007,No Info,Pool,"$255,000",Las Vegas
997,296 Molinetto St,3 Beds,3 Baths,"2,095 sqft",2019,2 Car Garage,Yes,"$995,000",Las Vegas
998,816 Trotter Cir,5 Beds,5 Baths,"5,357 sqft (on 0.35 acres)",1989,Garage,Yes,"$1,995,000",Las Vegas


In [5]:
df.to_csv('df_las_vegas.csv', index=False)